In [1]:
import sqlite3

In [2]:
sqlite3.version #package 의 버전.

'2.6.0'

In [3]:
sqlite3.sqlite_version # 실제 돌아가는 cursor 의 버전


'3.21.0'

In [4]:
conn = sqlite3.connect(':memory:')

In [5]:
cur  = conn.cursor()

In [6]:
cur.execute('create table people (name_last,age)')

In [7]:
who = "Yeltsin"
age = 72
cur.execute('insert into people values (?,?)',(who,age))

In [8]:
result = cur.execute('select * from people')

In [9]:
result

In [10]:
cur.execute('select * from people where name_last=:who and age=:age',{'who':who,'age':age})

In [11]:
print(cur.fetchone())

('Yeltsin', 72)


In [12]:
cur.fetchone()

## insert 할때 , 두가지 방식 qmark 랑 named 방식. 

In [13]:
sql = "insert into people values (?,?)"

In [14]:
type(sql)

str

In [15]:
data = [ ('kim',25),('lee',24),('lim',44)]

In [16]:
cur.executemany(sql,data) # 이 sql 문을 데이터 크기만큼 반복해준다 

In [17]:
cur.execute('select * from people where name_last=:who and age=:age',{'who':who,'age':age})

In [18]:
print(cur.fetchone())

('Yeltsin', 72)


In [19]:
#script로 작성하기

In [20]:
cur.executescript("""
    create table person(
        first_name text primary key,
        last_name text not null
        );
    insert into person values('name','kim');
""")

In [21]:
cur.execute('select * from people')

In [22]:
print(cur.fetchone())

('Yeltsin', 72)


In [23]:
conn.close()

## 새로운 DB 를 만들기

In [51]:
conn=sqlite3.connect("test7.db")

In [52]:
cur = conn.cursor()

In [53]:
conn.execute('''
    CREATE TABLE COMPANY(
        ID INT PRIMARY KEY NOT NULL,
        NAME TEXT NOT NULL,
        AGE INT NOT NULL,
        ADDRESS CHAR(50),
        SALARY REAL);
        ''')

In [54]:
conn.execute("insert into company (id, name, age, address, salary) values (1, 'Paul', 32, 'California', 20000.00)");
conn.execute("insert into company (id, name, age, address, salary) values (:id, :name, :age, :address, :salary)",
            {'id':2, 'name':'Allen', 'age':25, 'address':'Texas', 'salary':15000.00});
data = [(3, 'Teddy', 23, 'Norway', 200000.00), (4, 'Mark', 25, 'Rich-Mond', 65000.00)]

conn.executemany("insert into company (id, name, age, address, salary) values (?,?,?,?,?)" ,data);

conn.executescript("""
insert into company (id, name, age, address, salary) values (5, 'Mark1', 25, 'Rich-Mond', 65000.00);
insert into company (id, name, age, address, salary) values (6, 'Mark2', 25, 'Rich-Mond', 65000.00);
""");

conn.commit()

In [55]:
type(conn)

sqlite3.Connection

In [57]:
temp = conn.execute("select * from company")

In [58]:
type(temp) # execute 가 커서를 가지고온것 자동으로

sqlite3.Cursor

In [60]:
for row in temp:
    print(type(row),row)

결과적으로  fetch 한것과 같은결과가나온다.

In [61]:
conn.execute("update company set salary = 250300.00 where ID = :id",{"id":1}) # parameter 로 네임드방식은 dictionary 를 넣어야한다.

In [62]:
#update 의 경우 기존의 데이터를 변경하는것이기 때문에 무결성의 원칙에 맞는지 확인해야한다
#따라서 commit() 으로 트랜잭션을 끝내주어야 하면 그때 DB 에 적용될수있다.
#select를 하기전에 commit 을 한다. 하지만 commit 이 너무 많아지면 오버헤드발생
 

In [63]:
conn.execute('delete from company where Id =2 ')

In [64]:
cursor = conn.execute('select * from company')

In [65]:
for row in cursor:
    print(row)

(1, 'Paul', 32, 'California', 250300.0)
(3, 'Teddy', 23, 'Norway', 200000.0)
(4, 'Mark', 25, 'Rich-Mond', 65000.0)
(5, 'Mark1', 25, 'Rich-Mond', 65000.0)
(6, 'Mark2', 25, 'Rich-Mond', 65000.0)


# 얘가 가지고있는 메모리에서는 2가 사라져있지만, 실제 디비에서는 그렇지않다 .

#commit 을 해주어야한다 . 메모리단에서만 다루다가 ,실제로 디비에 반영해주기위해서 ,  Sqlite for webrowser 를 통해 확인하였다


## 실습시간 

In [66]:
import sqlite3 as sql
con =None
try:
    con = sql.connect('test.db')
    cur = con.cursor()
    cur.execute('select sqlite_version()')
    data = cur.fetchone()
    print(type(data))
except Exception as e:
    print("Error: ", e)
finally:
    if con:
        con.close()

<class 'tuple'>


In [70]:
con = sql.connect('test8.db')
with con:
    cur = con.cursor()
    cur.execute("CREATE TABLE Cars(Id INT PRIMARY KEY, Name TEXT, Price INT)")
    cur.execute("INSERT INTO Cars VALUES(1,'Audi',52343)")
    cur.execute("INSERT INTO Cars VALUES(2,'Mercedes',53511)")
    cur.execute("INSERT INTO Cars VALUES(3,'Skoda',5999)")

In [71]:
cars = [
    (1,'Audi',52343),
    (2,'Mercedes',53511),
    (3,'Skoda',5999),
    (4,'Volvo',29000),
    (5,'Hummer',50000)
]

In [72]:
con =sql.connect('test4.db')

In [74]:
with con:
    cur = con.cursor()
    cur.execute("DROP TABLE IF EXISTS Cars")
    cur.execute("CREATE TABLE Cars(Id INT PRIMARY KEY,Name TEXT)")

OperationalError: database is locked

In [75]:
con  = sql.connect(':memory:')
with con:
    cur = con.cursor()
    cur.execute("create table friends (id integer primary key , name text);")
    cur.execute("insert into friends(name)values('Tom');")
    cur.execute("insert into friends(name)values('rebecca');")
    cur.execute("insert into friends(name)values('Jim');")
    
    lid = cur.lastrowid
    print(lid)
    print(type(cur.execute('select * from friends')))
    print(cur.fetchall()[2])
    

3
<class 'sqlite3.Cursor'>
(3, 'Jim')


In [76]:
con = sql.connect('test3.db')

In [77]:
with con: # with 를 사용하면 .  끝나면 con 이 사라진다.
    cur = con.cursor()
    cur.execute('select * from cars')
    
    while True:
        row = cur.fetchone()
        if row == None : 
            break
    print(row[0],row[1],row[2])

TypeError: 'NoneType' object is not subscriptable

In [78]:
cur.fetchall()

[]